# Smart Advertisement Service


## Business scenario

An advertising consultant startup in US, which focuses on consulting range of business on effective advertisement, wants to adapt the cutting edge technology in order to enhance their quality of services. Lately conducted research on company has found out that around 14 percent of the national population are immigrant, [source](https://www.americanimmigrationcouncil.org/research/immigrants-in-the-united-states) . Thus the company has decided to build a data warehouse which will used for analytics for better consultation on advertisement. Not limited to that, the warehouse will also be used as a brain for backend services that will possibly automate the advertisement of certain category.

## Structure of the Project

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

### Step 1: Scope the Project and Gather Data

#### Scope
In this project I will be create a cloud data warehouse that will support answering question through analytics tables and dashboards. Later, the warehouse could be used as a backend for developing automatic advertisement services. 

The following steps will be carried out:

The data I
Explain what you plan to do in the project in more detail. What data do you use? What is your end solution look like? What tools did you use? etc.

#### The data

The project uses data from two data sources
- I94 immigration data that comes from the US National Tourism and Trade Office.
- U.S City demographic data that comes from Opensoft.
- World temperature data from Kaggle


### Architecture

#### Describe and Gather Data 
Describe the data sets you're using. Where did it come from? What type of information is included? 



In [1]:
# Do all imports and installs here
import pandas as pd
import os
from pyspark.sql import SparkSession
from pyspark.sql import types as T
from pyspark.sql import functions as F
from pyspark.sql.types import IntegerType
from datetime import datetime, timedelta
import json
from pyspark.sql.functions import desc, monotonically_increasing_id, udf, to_date, from_unixtime, trim, col

In [2]:
# Create spark session
spark = SparkSession.builder.config(
    "spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0"
).getOrCreate()

21/12/06 22:59:51 WARN Utils: Your hostname, Yugeshs-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.0.18 instead (on interface en0)
21/12/06 22:59:51 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/Users/yugesh/opt/anaconda3/envs/airflow/lib/python3.8/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/yugesh/.ivy2/cache
The jars for the packages stored in: /Users/yugesh/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-60f082a5-fd0f-4089-b3bd-7d1e3333380c;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;2.7.0 in central
	found org.apache.hadoop#hadoop-common;2.7.0 in central
	found org.apache.hadoop#hadoop-annotations;2.7.0 in central
	found com.google.guava#guava;11.0.2 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found commons-cli#commons-cli;1.2 in central
	found org.apache.commons#commons-math3;3.1.1 in central
	found xmlenc#xmlenc;0.52 in central
	found commons-httpclient#commons-httpclient;3.1 in central
	found commons-logging#commons-logging;1.1.3 in central
	found commons-codec#commons-codec;1.4 in central
	found commons-io#commons-io;2.4 in central
	found commons-net#commons-net;3.1 in central
	found commons-collections#commons-colle

In [3]:
# Read in the data here
# partial
# immigration_df = spark.read.parquet("./data/sas_data/part-00000-b9542815-7a8d-45fc-9c67-c9c5007ad0d4-c000.snappy.parquet")
# all
immigration_df = spark.read.parquet("./data/sas_data/")


In [4]:
immigration_df.count()

3096313

### Step 2: Explore and Assess the Data
#### Explore the Data 
Identify data quality issues, like missing values, duplicate data, etc.

#### Cleaning Steps
Document steps necessary to clean the data

### Exploring immigration data

In [5]:
immigration_df.printSchema()

root
 |-- cicid: double (nullable = true)
 |-- i94yr: double (nullable = true)
 |-- i94mon: double (nullable = true)
 |-- i94cit: double (nullable = true)
 |-- i94res: double (nullable = true)
 |-- i94port: string (nullable = true)
 |-- arrdate: double (nullable = true)
 |-- i94mode: double (nullable = true)
 |-- i94addr: string (nullable = true)
 |-- depdate: double (nullable = true)
 |-- i94bir: double (nullable = true)
 |-- i94visa: double (nullable = true)
 |-- count: double (nullable = true)
 |-- dtadfile: string (nullable = true)
 |-- visapost: string (nullable = true)
 |-- occup: string (nullable = true)
 |-- entdepa: string (nullable = true)
 |-- entdepd: string (nullable = true)
 |-- entdepu: string (nullable = true)
 |-- matflag: string (nullable = true)
 |-- biryear: double (nullable = true)
 |-- dtaddto: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- insnum: string (nullable = true)
 |-- airline: string (nullable = true)
 |-- admnum: double (nullable = 

#### Convert double to integer

In [6]:
int_cols = ["cicid", "i94cit", "i94res", "arrdate", "i94mode", "depdate", "i94bir", "i94visa"]
# Additional options "dtadfile", "daddto"

for col_name in int_cols:
    immigration_df = immigration_df.withColumn(col_name, immigration_df[col_name].cast(IntegerType()))

immigration_df.select(int_cols).show(1)

+-------+------+------+-------+-------+-------+------+-------+
|  cicid|i94cit|i94res|arrdate|i94mode|depdate|i94bir|i94visa|
+-------+------+------+-------+-------+-------+------+-------+
|5748517|   245|   438|  20574|      1|  20582|    40|      1|
+-------+------+------+-------+-------+-------+------+-------+
only showing top 1 row



#### Remove duplicate values

- Drop duplicate with original ccid, however unable to remove duplicate because of ccid.
- Drop duplicate without ccid and recreate ccid able to remove ()

In [7]:
# Dropping duplicate with ccid
with_duplicate = immigration_df.count()

immigration_df.dropDuplicates()

without_duplicate = immigration_df.count()
print(f"Total number of deleted duplicate values {with_duplicate - without_duplicate}")
print(f"Total number of availabel records {immigration_df.count()}")

Total number of deleted duplicate values 0
Total number of availabel records 3096313


In [8]:
# Dropping duplicate by excluding ccid

with_duplicate = immigration_df.count()

immigration_df = immigration_df.drop("cicid")
immigration_df = immigration_df.dropDuplicates()

immigration_df = immigration_df.withColumn("cicid", monotonically_increasing_id())

without_duplicate = immigration_df.count()
print(f"Total number of deleted duplicate values {with_duplicate - without_duplicate}")
print(f"Total number of available records {immigration_df.count()}")


21/12/06 23:00:15 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


Total number of deleted duplicate values 0


Total number of available records 3096313


#### Assign 0 to all null in integer column

In [9]:
immigration_df = immigration_df.fillna(0, int_cols)

#### Assign real values

In [10]:
def assign_value(col_name, key):
    ROOT = "./lookup/"
    key = str(key)
    col_name = col_name.lower()

    filepath = os.path.join(ROOT, f"{col_name}.json")
    with open(filepath, "r") as f:
        data = json.load(f)

    if col_name == "i94port":
        return data[key].split(",")[0] if key in data else None

    return data[key] if key in data else None

In [11]:
# Retrieve transporation mode using i94mode
get_mode_udf = udf(lambda x: assign_value(key=x, col_name="i94mode"), T.StringType())
immigration_df = immigration_df.withColumn("transportation_mode", get_mode_udf(immigration_df.i94mode))

In [12]:
# Retrieve arrived city
get_city_udf = udf(lambda x: assign_value(key=x, col_name="i94port"), T.StringType())
immigration_df = immigration_df.withColumn("arrived_city", get_city_udf(immigration_df.i94port))

In [13]:
# Retrieve arrived state
get_state_udf = udf(lambda x: assign_value(key=x, col_name="i94addr"), T.StringType())
immigration_df = immigration_df.withColumn("us_address", get_state_udf(immigration_df.i94addr))

In [14]:
# Retrieve origin city and travelled from using i94CIT and i94res
get_origin_udf = udf(lambda x: assign_value(key=x, col_name="i94cit"), T.StringType())
immigration_df = immigration_df.withColumn("origin_city", get_origin_udf(immigration_df.i94cit)).withColumn("traveled_from", get_origin_udf(immigration_df.i94res))

In [15]:
# Retrive i94visa with value
get_visa_udf = udf(lambda x: assign_value(key=x, col_name="i94visa"), T.StringType())
immigration_df = immigration_df.withColumn("visa_status", get_visa_udf(immigration_df.i94visa))

In [17]:
unused_cols = [ "i94yr","i94mon","count", "fltno",  "insnum", "entdepd", "biryear", "dtadfile", "biryear", "visapost", "entdepu", "admnum", "i94cit", "i94res", "i94port", "i94addr", "i94mode", "i94visa", "entdepa", "dtaddto"]
print(f"No of columns before removing: {len(immigration_df.columns)}")
immigration_df = immigration_df.drop(*unused_cols)
print(f"No of columns after removing: {len(immigration_df.columns)}")

No of columns before removing: 15
No of columns after removing: 15


In [18]:
immigration_df.printSchema()

root
 |-- arrdate: integer (nullable = true)
 |-- depdate: integer (nullable = true)
 |-- i94bir: integer (nullable = true)
 |-- occup: string (nullable = true)
 |-- matflag: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- airline: string (nullable = true)
 |-- visatype: string (nullable = true)
 |-- cicid: long (nullable = false)
 |-- transportation_mode: string (nullable = true)
 |-- arrived_city: string (nullable = true)
 |-- us_address: string (nullable = true)
 |-- origin_city: string (nullable = true)
 |-- traveled_from: string (nullable = true)
 |-- visa_status: string (nullable = true)



In [19]:
# Rename columns
immigration_df = immigration_df.withColumnRenamed("arrdate", "arrival_date").withColumnRenamed("depdate", "departure_date").withColumnRenamed("i94bir", "age").withColumnRenamed("occup", "occupation")

In [20]:
immigration_df = immigration_df.select(["cicid", "origin_city", "traveled_from", "arrived_city", "us_address", "arrival_date", "departure_date", "transportation_mode", "age", "gender",
"visa_status", "occupation", "airline"])

In [21]:
# test.select("visa_status").distinct().show()

#### Convert sas date format to "YYYY_MM_DD"

In [22]:
def convert_datetime(x):
    try:
        start = datetime(1960, 1, 1)
        return start + timedelta(days=int(x))
    except:
        return None
udf_datetime_from_sas = udf(lambda x: convert_datetime(x), T.DateType())

In [23]:
date_format = "%Y-%m-%d"
date_cols = ["arrdate", "depdate"]
convert_sas_udf = udf(lambda x: x if x is None else (timedelta(days=x) + datetime(1960, 1, 1)).strftime(date_format))

In [25]:
immigration_df = immigration_df.withColumn("arrival_date", udf_datetime_from_sas(immigration_df.arrival_date)).withColumn("departure_date", udf_datetime_from_sas(immigration_df.departure_date))
immigration_df.select(["arrival_date", "departure_date"]).show(5)

+------------+--------------+
|arrival_date|departure_date|
+------------+--------------+
|  2016-04-07|    1960-01-01|
|  2016-04-10|    1960-01-01|
|  2016-04-30|    1960-01-01|
|  2016-04-15|    2016-04-16|
|  2016-04-21|    2016-04-25|
+------------+--------------+
only showing top 5 rows



### Exploring us cities demographics

In [ ]:
filepath = "./data/us-cities-demographics.csv"
demographic_df = spark.read.csv(filepath,inferSchema=True, header=True, sep=';')

In [ ]:
demographic_df.printSchema()

#### Removing duplicates: Pivoting column

In [ ]:
# Finding types of races
demographic_df.select("race").distinct().show()

In [ ]:
# Pivot column Race to different columns
pivot_cols = ["City", "State"]
pivot_df = demographic_df.groupBy(pivot_cols).pivot("Race").sum("Count")

# Joining the pivot 
demographic_df = demographic_df.join(other=pivot_df, on=pivot_cols)

In [ ]:
demographic_df.printSchema()

#### Remove unwanted columns from the schema

In [ ]:
del_cols = ["median age", "Number of Veterans", "foreign-born", "Average Household Size", "State Code", "race", "count"]
demographic_df = demographic_df.drop(*del_cols)
demographic_df.show(1)

#### Dropping duplicates 

In [ ]:
# Dropping duplicate by excluding ccid

with_duplicate = demographic_df.count()

demographic_df = demographic_df.dropDuplicates()

without_duplicate = demographic_df.count()
print(f"Total number of deleted duplicate values {with_duplicate - without_duplicate}")
print(f"Total number of availabel records {demographic_df.count()}")

In [ ]:
demographic_df.printSchema()

#### Convert column names

In [ ]:
demographic_df = demographic_df.toDF("city", "state", "male_population", "female_population", "total_population", "american_indian_alaska_native", "asian", "black_african_american", "hispanic_latino", "white")

In [ ]:
demographic_df.printSchema()

#### Filling null with 0

In [ ]:
num_cols = ["male_population", "female_population", "total_population",  "american_indian_alaska_native", "asian", "black_african_american", "hispanic_latino", "white"]
demographic_df = demographic_df.fillna(0, num_cols)

#### Add id column

In [ ]:
demographic_df = demographic_df.withColumn("id", monotonically_increasing_id())
demographic_df.printSchema()

#### Reordering the columns

In [ ]:
demographic_df = demographic_df.select(["id", "city", "state", "american_indian_alaska_native",
"asian",
"black_african_american",
"hispanic_latino", "white", "male_population", "female_population", "total_population"])

In [ ]:
demographic_df.printSchema()

### Exploring airport data

In [ ]:
# reading csv file
airport_csv_path = "./data/airport-codes_csv.csv"
airport_df = spark.read.csv(airport_csv_path, header=True, sep=',', ignoreTrailingWhiteSpace=False)

airport_df.printSchema()

#### Removing whitespace in column name

In [ ]:
airport_df = airport_df.select([F.col(col).alias(col.replace(' ', '')) for col in airport_df.columns])
airport_df.show(1)

#### Excluding airport outside of US

In [ ]:
airport_df = airport_df.filter(airport_df.iso_country.like("%US%"))
airport_df.count()

#### Add state_code column

In [ ]:
get_state_udf = udf(lambda x :  x if x is None else x.split("-")[1])
airport_df = airport_df.withColumn("state_code", get_state_udf("iso_region"))

#### Remove unwanted columns

In [ ]:
airport_delete_cols = ["elevation_ft", "continent", "iso_country", "iso_region" , "gps_code", "iata_code", "local_code", "coordinates"]
airport_df = airport_df.drop(*airport_delete_cols)
airport_df.printSchema()

#### Drop duplicates

In [ ]:
# Dropping duplicate by excluding ident

with_duplicate = airport_df.count()

airport_df = airport_df.drop("ident")
airport_df = airport_df.dropDuplicates()

airport_df = airport_df.withColumn("id", monotonically_increasing_id())

without_duplicate = airport_df.count()
print(f"Total number of deleted values {with_duplicate - without_duplicate}")
print(f"Total number of available records {airport_df.count()}")

In [ ]:
airport_df.printSchema()

#### Reordering columns

In [ ]:
airport_df = airport_df.select(["id", "name", "type", "state_code", "municipality"])
airport_df.printSchema()

# Testing

In [ ]:
immigration_df.filter(immigration_df.i94visa > 1).show(20)

In [ ]:
airport_df.select("type").distinct().show()

In [ ]:
immigration_df.select("i94mode").distinct().show()

In [ ]:
immigration_df.select("i94visa").distinct().show()

In [ ]:
demographic_df.select("city").distinct().count()

In [ ]:
immigration_df.select("matflag").distinct().show()


In [ ]:
demographic_df.groupBy("city").count().sort(col("count").desc()).show()


In [ ]:
demographic_df.filter(demographic_df.city == "Columbia").show()

### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
Map out the conceptual data model and explain why you chose that model

#### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

In [ ]:
# Write code here

#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

In [ ]:
# Perform quality checks here

#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.